In [1]:
import os
os.chdir('/home/a001/Documents/ZhengHaoyu/python/1_important/OCMN')
import glob
from utils.generator import Generator, ERGenerator, BAGenerator, SFGenerator
from utils.utils import save_network, read_network, setup_logger, create_output_file
from utils.plot import transform_graphs, visualize_networks_with_bipartite, visualize_networks_with_matching_and_bipartite
from matching import Matching, MultiMatching
import config
import datetime
import numpy as np
import pandas as pd
import copy
from tqdm import tqdm
import networkx as nx
import copy
import time

# Generate Synthetic Networks

In [ ]:
networks = ["base"] + [f"overlap={overlap}" for overlap in config.NETWORK_OVERLAP]

def generate_and_save_networks(k_range={"start": 2, "end": 10, "step": 0.1}):
    for k in tqdm(np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"])):
        for n in config.NETWORK_NODES_LIST:
            k = round(k, 2)
            
            er_generator = ERGenerator(n, k)
            er_graphs = er_generator.generate_networks(len(networks), config.NETWORK_OVERLAP)
            dir = f"ER_n={n}_k={k}"
            os.makedirs(os.path.join(config.SYNTHETIC_NET_PATH, 'ER', dir), exist_ok=True)
            for i in range(len(networks)):
                save_network(er_graphs[i], os.path.join(config.SYNTHETIC_NET_PATH, 'ER', dir, f"{networks[i]}.txt"))
            
            ba_generator = SFGenerator(n, k)
            ba_graphs = ba_generator.generate_networks(len(networks), config.NETWORK_OVERLAP)
            dir = f"BA_n={n}_k={k}"
            os.makedirs(os.path.join(config.SYNTHETIC_NET_PATH, 'BA', dir), exist_ok=True)
            for i in range(len(networks)):
                save_network(ba_graphs[i], os.path.join(config.SYNTHETIC_NET_PATH, 'BA', dir, f"{networks[i]}.txt"))

generate_and_save_networks()

100%|██████████| 81/81 [07:19<00:00,  5.42s/it]


# 不同网络类型、不同平均度k、不同节点数N、重复实验

In [2]:
def optimization_amount(
    k_range={"start": 2, "end": 10, "step": 0.1},
    n_list=config.NETWORK_NODES_LIST,
    result_columns=["network_type", "N", "<k>", "seq", "MDS_1", "MDS_2", "Diff_MDS_1", "Diff_MDS_2", "UMDS_0", "UMDS_1", "UMDS_2", "average_depth", "time_1", "time_2"]
):
    output_file_name = create_output_file(result_columns, "optimization_amount")

    for n in n_list:
        print(f"Processing n={n}...")
    
        for network_type in ["ER", "BA"]:
            for k in tqdm(np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"])):
                k = round(k, 2)
                for seq in range(10):
                    dir = f"{network_type}_n={n}_k={k}"
                    matchings = []
                    for file in os.listdir(os.path.join(config.SYNTHETIC_NET_PATH, network_type, dir)):
                        # 包含base或-1的文件名
                        if "base" in file or "-1" in file:
                            graph = read_network(os.path.join(config.SYNTHETIC_NET_PATH, network_type, dir, file), n)
                            matching = Matching(graph)
                            matching.HK_algorithm()
                            matching.find_all_alternating_reachable_set()
                            matchings.append(matching)
                    
                    multi_matching = MultiMatching(matchings)
                    baseline_matching = copy.deepcopy(multi_matching)

                    start_time = time.time()
                    pre_diff_mds_1_size, pre_diff_mds_2_size, pre_union_size, union_size, average_depth = multi_matching.MOUI()
                    end_time = time.time()
                    time_2 = end_time - start_time
                    
                    start_time = time.time()
                    min_union_size = baseline_matching.RRMU()
                    end_time = time.time()
                    time_1 = end_time - start_time
                    
                    with open(output_file_name, "a", encoding="utf-8") as output_file:
                        output_file.write(",".join([
                            f"{network_type}+{network_type}", str(n), str(k), str(seq), 
                            str(len(matchings[0].driver_nodes)), str(len(matchings[1].driver_nodes)), 
                            str(pre_diff_mds_1_size), str(pre_diff_mds_2_size), 
                            str(pre_union_size), str(min_union_size), str(union_size), str(average_depth),
                            str(time_1), str(time_2)
                        ]) + "\n")
        
        for k in tqdm(np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"])):
            k = round(k, 2)
            for seq in range(10):
                matchings = []
                graph_1 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, "ER", f"ER_n={n}_k={k}", f"base.txt"), n)
                matching_1 = Matching(graph_1)
                matching_1.HK_algorithm()
                matching_1.find_all_alternating_reachable_set()
                matchings.append(matching_1)
                graph_2 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, "BA", f"BA_n={n}_k={k}", f"base.txt"), n)
                matching_2 = Matching(graph_2)
                matching_2.HK_algorithm()
                matching_2.find_all_alternating_reachable_set()
                matchings.append(matching_2)

                multi_matching = MultiMatching(matchings)
                baseline_matching = copy.deepcopy(multi_matching)
                
                start_time = time.time()
                pre_diff_mds_1_size, pre_diff_mds_2_size, pre_union_size, union_size, average_depth = multi_matching.MOUI()
                end_time = time.time()
                time_2 = end_time - start_time
                
                start_time = time.time()
                min_union_size = baseline_matching.RRMU()
                end_time = time.time()
                time_1 = end_time - start_time
                with open(output_file_name, "a", encoding="utf-8") as output_file:
                    output_file.write(",".join([
                        f"ER+BA", str(n), str(k), str(seq), 
                        str(len(matching_1.driver_nodes)), str(len(matching_2.driver_nodes)), 
                        str(pre_diff_mds_1_size), str(pre_diff_mds_2_size), 
                        str(pre_union_size), str(min_union_size), str(union_size), str(average_depth),
                        str(time_1), str(time_2)
                    ]) + "\n")

optimization_amount()

Processing n=1000...


100%|██████████| 81/81 [13:43<00:00, 10.17s/it]


# 不同网络类型、不同平均度k、不同网络重叠程度

In [3]:
def optimization_proportion(
    k_range={"start": 2, "end": 10, "step": 0.1},
    n_list=config.NETWORK_NODES_LIST,
    overlap_list=config.NETWORK_OVERLAP,
    result_columns=["network_type", "N", "<k>", "overlap", "MDS_1", "MDS_2", "Diff_MDS_1", "Diff_MDS_2", "UMDS_0", "UMDS_1", "UMDS_2", "average_depth", "time_1", "time_2"]
):
    output_file_name = create_output_file(result_columns, "optimization_proportion")
    
    for n in n_list:
        print(f"Processing n={n}...")
        for network_type in ["ER", "BA"]:
            print(f"\tProcessing {network_type}...")
            for overlap in overlap_list:
                overlap = round(overlap, 3)
                if overlap < 0:
                    continue
                for k in tqdm(np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"]), desc=f"\t\tOverlap={overlap}"):
                    k = round(k, 2)
                    dir = f"{network_type}_n={n}_k={k}"

                    matchings = []
                    for file in os.listdir(os.path.join(config.SYNTHETIC_NET_PATH, network_type, dir)):
                        if "base" in file or f"overlap={overlap}.txt" in file:
                            graph = read_network(os.path.join(config.SYNTHETIC_NET_PATH, network_type, dir, file), n)
                            matching = Matching(graph)
                            matching.HK_algorithm()
                            matching.find_all_alternating_reachable_set()
                            matchings.append(matching)
                    
                    multi_matching = MultiMatching(matchings)
                    baseline_matching = copy.deepcopy(multi_matching)

                    start_time = time.time()
                    pre_diff_mds_1_size, pre_diff_mds_2_size, pre_union_size, union_size, average_depth = multi_matching.MOUI()
                    end_time = time.time()
                    time_2 = end_time - start_time
                    
                    start_time = time.time()
                    min_union_size = baseline_matching.RRMU()
                    end_time = time.time()
                    time_1 = end_time - start_time

                    with open(output_file_name, "a", encoding="utf-8") as output_file:
                        output_file.write(",".join([
                            f"{network_type}+{network_type}", str(n), str(k), str(overlap),
                            str(len(matchings[0].driver_nodes)), str(len(matchings[1].driver_nodes)), 
                            str(pre_diff_mds_1_size), str(pre_diff_mds_2_size), 
                            str(pre_union_size), str(min_union_size), str(union_size), str(average_depth),
                            str(time_1), str(time_2)
                        ]) + "\n")

optimization_proportion()

Processing n=1000...
	Processing ER...


		Overlap=0.9: 100%|██████████| 81/81 [01:26<00:00,  1.07s/it]


	Processing BA...


		Overlap=0.9: 100%|██████████| 81/81 [01:19<00:00,  1.01it/s]


# Homogeneous Networks

In [ ]:
def same_structure(
        k_range={"start": 1, "end": 6, "step": 0.1},
        gamma_range={"start": 2, "end": 5, "step": 0.1},
        result_columns=["type", "n", "k", "gamma", "seq", "MDS1", "MDS2", "pre_intersection", "intersection", "pre_union", "union"]):
    output_file_name = create_output_file(result_columns, "same_structure")

    n = 1000

    type = "ER"
    for k in tqdm(np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"])):
        k = round(k, 2)
        for seq in range(10):
            dir = f"{type}_n={n}_k={k}"
            matchings = []
            for file in os.listdir(os.path.join(config.SYNTHETIC_NET_PATH, type, dir)):
                graph = read_network(os.path.join(config.SYNTHETIC_NET_PATH, type, dir, file), n)
                matching = Matching(graph)
                matching.HK_algorithm()
                matching.find_all_alternating_reachable_set()
                matchings.append(matching)
            
            multi_matching = MultiMatching(matchings)
            pre_intersection_size, intersection_size, pre_union_size, union_size = multi_matching.MOUI()
            with open(output_file_name, "a", encoding="utf-8") as output_file:
                output_file.write(",".join([
                    f"{type}+{type}", str(n), str(k), '', str(seq), 
                    str(len(matchings[0].driver_nodes)), str(len(matchings[1].driver_nodes)), 
                    str(pre_intersection_size), str(intersection_size), str(pre_union_size), str(union_size)
                ]) + "\n")
    
    type = "SF"
    gamma = round(3.0, 2)
    for k in tqdm(np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"])):
        k = round(k, 2)
        for seq in range(10):
            dir = f"{type}_n={n}_k={k}_gamma={gamma}"
            matchings = []
            for file in os.listdir(os.path.join(config.SYNTHETIC_NET_PATH, type, dir)):
                graph = read_network(os.path.join(config.SYNTHETIC_NET_PATH, type, dir, file), n)
                matching = Matching(graph)
                matching.HK_algorithm()
                matching.find_all_alternating_reachable_set()
                matchings.append(matching)
            
            multi_matching = MultiMatching(matchings)
            pre_intersection_size, intersection_size, pre_union_size, union_size = multi_matching.MOUI()
            with open(output_file_name, "a", encoding="utf-8") as output_file:
                output_file.write(",".join([
                    f"{type}+{type}_gamma={gamma}", str(n), str(k), str(gamma), str(seq), 
                    str(len(matchings[0].driver_nodes)), str(len(matchings[1].driver_nodes)), 
                    str(pre_intersection_size), str(intersection_size), str(pre_union_size), str(union_size)
                ]) + "\n")

    k = round(3.0, 2)
    for gamma in tqdm(np.arange(gamma_range['start'], gamma_range['end'] + gamma_range["step"], gamma_range["step"])):
        gamma = round(gamma, 2)
        for seq in range(10):
            dir = f"{type}_n={n}_k={k}_gamma={gamma}"
            matchings = []
            for file in os.listdir(os.path.join(config.SYNTHETIC_NET_PATH, type, dir)):
                graph = read_network(os.path.join(config.SYNTHETIC_NET_PATH, type, dir, file), n)
                matching = Matching(graph)
                matching.HK_algorithm()
                matching.find_all_alternating_reachable_set()
                matchings.append(matching)
            
            multi_matching = MultiMatching(matchings)
            pre_intersection_size, intersection_size, pre_union_size, union_size = multi_matching.MOUI()
            with open(output_file_name, "a", encoding="utf-8") as output_file:
                output_file.write(",".join([
                    f"{type}+{type}_k={k}", str(n), str(k), str(gamma), str(seq), 
                    str(len(matchings[0].driver_nodes)), str(len(matchings[1].driver_nodes)), 
                    str(pre_intersection_size), str(intersection_size), str(pre_union_size), str(union_size)
                ]) + "\n")

same_structure()

  0%|          | 0/51 [00:00<?, ?it/s]

100%|██████████| 31/31 [00:21<00:00,  1.43it/s]


# Heterogeneous Networks

In [ ]:
def same_structure_diff_k_or_gamma(
        k_range={"start": 1, "end": 6, "step": 0.1},
        gamma_range={"start": 2, "end": 5, "step": 0.1}, 
        result_columns=["type", "n", "k1", "k2", "gamma1", "gamma2", "MDS1", "MDS2", "pre_intersection", "intersection", "pre_union", "union"]):
    output_file_name = create_output_file(result_columns, "same_structure_diff_k_or_gamma")

    n = 1000

    type = "ER"
    for k1 in tqdm(np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"])):
        k1 = round(k1, 2)
        dir1 = f"{type}_n={n}_k={k1}"
        for k2 in np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"]):
            k2 = round(k2, 2)
            dir2 = f"{type}_n={n}_k={k2}"

            matchings = []
            graph1 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, type, dir1, "1.txt"), n)
            matching1 = Matching(graph1)
            matching1.HK_algorithm()
            matching1.find_all_alternating_reachable_set()
            matchings.append(matching1)
            graph2 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, type, dir2, "2.txt"), n)
            matching2 = Matching(graph2)
            matching2.HK_algorithm()
            matching2.find_all_alternating_reachable_set()
            matchings.append(matching2)

            multi_matching = MultiMatching(matchings)
            pre_intersection_size, intersection_size, pre_union_size, union_size = multi_matching.MOUI()
            with open(output_file_name, "a", encoding="utf-8") as output_file:
                output_file.write(",".join([
                    f"{type}+{type}", str(n), str(k1), str(k2), '', '', 
                    str(len(matching1.driver_nodes)), str(len(matching2.driver_nodes)), 
                    str(pre_intersection_size), str(intersection_size), str(pre_union_size), str(union_size)
                ]) + "\n")
        
    type = "SF"
    k = round(3.0, 2)
    for gamma1 in tqdm(np.arange(gamma_range['start'], gamma_range['end'] + gamma_range["step"], gamma_range["step"])):
        gamma1 = round(gamma1, 2)
        dir1 = f"{type}_n={n}_k={k}_gamma={gamma1}"
        for gamma2 in np.arange(gamma_range['start'], gamma_range['end'] + gamma_range["step"], gamma_range["step"]):
            gamma2 = round(gamma2, 2)
            dir2 = f"{type}_n={n}_k={k}_gamma={gamma2}"

            matchings = []
            graph1 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, type, dir1, "1.txt"), n)
            matching1 = Matching(graph1)
            matching1.HK_algorithm()
            matching1.find_all_alternating_reachable_set()
            matchings.append(matching1)
            graph2 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, type, dir2, "2.txt"), n)
            matching2 = Matching(graph2)
            matching2.HK_algorithm()
            matching2.find_all_alternating_reachable_set()
            matchings.append(matching2)

            multi_matching = MultiMatching(matchings)
            pre_intersection_size, intersection_size, pre_union_size, union_size = multi_matching.MOUI()
            with open(output_file_name, "a", encoding="utf-8") as output_file:
                output_file.write(",".join([
                    f"{type}+{type}", str(n), str(k), str(k), str(gamma1), str(gamma2), 
                    str(len(matching1.driver_nodes)), str(len(matching2.driver_nodes)), 
                    str(pre_intersection_size), str(intersection_size), str(pre_union_size), str(union_size)
                ]) + "\n")
                
    gamma = round(3.0, 2)
    for k1 in tqdm(np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"])):
        k1 = round(k1, 2)
        dir1 = f"{type}_n={n}_k={k1}_gamma={gamma}"
        for k2 in np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"]):
            k2 = round(k2, 2)
            dir2 = f"{type}_n={n}_k={k2}_gamma={gamma}"

            matchings = []
            graph1 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, type, dir1, "1.txt"), n)
            matching1 = Matching(graph1)
            matching1.HK_algorithm()
            matching1.find_all_alternating_reachable_set()
            matchings.append(matching1)
            graph2 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, type, dir2, "2.txt"), n)
            matching2 = Matching(graph2)
            matching2.HK_algorithm()
            matching2.find_all_alternating_reachable_set()
            matchings.append(matching2)

            multi_matching = MultiMatching(matchings)
            pre_intersection_size, intersection_size, pre_union_size, union_size = multi_matching.MOUI()
            with open(output_file_name, "a", encoding="utf-8") as output_file:
                output_file.write(",".join([
                    f"{type}+{type}", str(n), str(k1), str(k2), str(gamma), str(gamma), 
                    str(len(matching1.driver_nodes)), str(len(matching2.driver_nodes)), 
                    str(pre_intersection_size), str(intersection_size), str(pre_union_size), str(union_size)
                ]) + "\n")

same_structure_diff_k_or_gamma()

 39%|███▉      | 20/51 [7:12:35<11:10:31, 1297.80s/it]


KeyboardInterrupt: 

# Hybrid Networks

In [ ]:
def diff_structure_diff_k(
        k_range={"start": 1, "end": 6, "step": 0.1},
        result_columns=["type", "n", "k1", "k2", "gamma1", "gamma2", "MDS1", "MDS2", "pre_intersection", "intersection", "pre_union", "union"]):
    output_file_name = create_output_file(result_columns, "diff_structure_diff_k")

    n = 1000
    
    gamma = round(3.0, 2)
    for k1 in tqdm(np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"])):
        k1 = round(k1, 2)
        dir1 = f"ER_n={n}_k={k1}"
        for k2 in np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"]):
            k2 = round(k2, 2)
            dir2 = f"SF_n={n}_k={k2}_gamma={gamma}"

            matchings = []
            graph1 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, 'ER', dir1, "1.txt"), n)
            matching1 = Matching(graph1)
            matching1.HK_algorithm()
            matching1.find_all_alternating_reachable_set()
            matchings.append(matching1)
            graph2 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, 'SF', dir2, "1.txt"), n)
            matching2 = Matching(graph2)
            matching2.HK_algorithm()
            matching2.find_all_alternating_reachable_set()
            matchings.append(matching2)

            multi_matching = MultiMatching(matchings)
            pre_intersection_size, intersection_size, pre_union_size, union_size = multi_matching.MOUI()
            with open(output_file_name, "a", encoding="utf-8") as output_file:
                output_file.write(",".join([
                    f"ER+SF", str(n), str(k1), str(k2), '', str(gamma), 
                    str(len(matching1.driver_nodes)), str(len(matching2.driver_nodes)), 
                    str(pre_intersection_size), str(intersection_size), str(pre_union_size), str(union_size)
                ]) + "\n")

diff_structure_diff_k()

100%|██████████| 51/51 [03:01<00:00,  3.56s/it]


In [ ]:
def diff_structure_diff_k_or_gamma(
        k_range={"start": 1, "end": 6, "step": 0.1},
        gamma_range={"start": 2, "end": 5, "step": 0.1}, 
        result_columns=["type", "n", "k1", "k2", "gamma1", "gamma2", "MDS1", "MDS2", "pre_intersection", "intersection", "pre_union", "union"]):
    output_file_name = create_output_file(result_columns, "diff_structure_diff_k_or_gamma")

    n = 1000

    for k in tqdm(np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"])):
        k = round(k, 2)
        dir1 = f"ER_n={n}_k={k}"
        for gamma in np.arange(gamma_range['start'], gamma_range['end'] + gamma_range["step"], gamma_range["step"]):
            gamma = round(gamma, 2)
            dir2 = f"SF_n={n}_k={k}_gamma={gamma}"

            matchings = []
            graph1 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, 'ER', dir1, "1.txt"), n)
            matching1 = Matching(graph1)
            matching1.HK_algorithm()
            matching1.find_all_alternating_reachable_set()
            matchings.append(matching1)
            graph2 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, 'SF', dir2, "1.txt"), n)
            matching2 = Matching(graph2)
            matching2.HK_algorithm()
            matching2.find_all_alternating_reachable_set()
            matchings.append(matching2)

            multi_matching = MultiMatching(matchings)
            pre_intersection_size, intersection_size, pre_union_size, union_size = multi_matching.MOUI()
            with open(output_file_name, "a", encoding="utf-8") as output_file:
                output_file.write(",".join([
                    f"ER+SF", str(n), str(k), str(k), '', str(gamma), 
                    str(len(matching1.driver_nodes)), str(len(matching2.driver_nodes)), 
                    str(pre_intersection_size), str(intersection_size), str(pre_union_size), str(union_size)
                ]) + "\n")

diff_structure_diff_k_or_gamma()

100%|██████████| 51/51 [01:40<00:00,  1.98s/it]


# Other Experiments

In [ ]:
def diff_n_type_k(
        k_range={"start": 1, "end": 8, "step": 0.1}, 
        result_columns=["type", "n", "k", "seq", "MDS1", "MDS2", "pre_intersection", "intersection", "pre_union", "union"]):
    output_file_name = create_output_file(result_columns)
    
    for k in np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"]):
        k = round(k, 2)
        for n in config.NETWORK_NODES_LIST:
            for seq in range(10):
                for type, _ in config.NETWORK_TYPES.items():
                    dir = f"{type}_n={n}_k={k}"
                    print(dir)
                    matchings = []
                    for file in os.listdir(os.path.join(config.SYNTHETIC_NET_PATH, dir)):
                        graph = read_network(os.path.join(config.SYNTHETIC_NET_PATH, dir, file), n)
                        matching = Matching(graph)
                        matching.HK_algorithm()
                        matching.find_all_alternating_reachable_set()
                        matchings.append(matching)
                    
                    multi_matching = MultiMatching(matchings)
                    pre_intersection_size, intersection_size, pre_union_size, union_size = multi_matching.MOUI()
                    with open(output_file_name, "a", encoding="utf-8") as output_file:
                        output_file.write(",".join([f"{type}+{type}", str(n), str(k), str(seq), str(len(matchings[0].driver_nodes)), str(len(matchings[1].driver_nodes)), 
                                                    str(pre_intersection_size), str(intersection_size), str(pre_union_size), str(union_size)]) + "\n")
                matchings = []
                for type, _ in config.NETWORK_TYPES.items():
                    dir = f"{type}_n={n}_k={k}"
                    print(dir)
                    graph = read_network(os.path.join(config.SYNTHETIC_NET_PATH, dir, "1.txt"), n)
                    matching = Matching(graph)
                    matching.HK_algorithm()
                    matching.find_all_alternating_reachable_set()
                    matchings.append(matching)
                
                multi_matching = MultiMatching(matchings)
                pre_intersection_size, intersection_size, pre_union_size, union_size = multi_matching.MOUI()
                with open(output_file_name, "a", encoding="utf-8") as output_file:
                    output_file.write(",".join([
                        f"ER+BA", str(n), str(k), str(seq), 
                        str(len(matchings[0].driver_nodes)), str(len(matchings[1].driver_nodes)), 
                        str(pre_intersection_size), str(intersection_size), str(pre_union_size), str(union_size)
                    ]) + "\n")

In [ ]:
def diff_n_gamma(
        gamma_range={"start": 2, "end": 8, "step": 0.1}, 
        result_columns=["type", "n", "k", "gamma", "seq", "MDS1", "MDS2", "pre_intersection", "intersection", "pre_union", "union"]):
    output_file_name = create_output_file(result_columns)
    
    type = "BA"
    k = round(3.0, 2)
    for gamma in np.arange(gamma_range['start'], gamma_range['end'] + gamma_range["step"], gamma_range["step"]):
        gamma = round(gamma, 2)
        for n in config.NETWORK_NODES_LIST:
            for seq in range(10):
                dir = f"{type}_n={n}_k={k}_gamma={gamma}"
                print(dir)
                matchings = []
                for file in os.listdir(os.path.join(config.SYNTHETIC_NET_PATH, 'BA', dir)):
                    graph = read_network(os.path.join(config.SYNTHETIC_NET_PATH, 'BA', dir, file), n)
                    matching = Matching(graph)
                    matching.HK_algorithm()
                    matching.find_all_alternating_reachable_set()
                    matchings.append(matching)
                
                multi_matching = MultiMatching(matchings)
                pre_intersection_size, intersection_size, pre_union_size, union_size = multi_matching.MOUI()
                with open(output_file_name, "a", encoding="utf-8") as output_file:
                    output_file.write(",".join([
                        f"{type}+{type}", str(n), str(k), str(gamma), str(seq), 
                        str(len(matchings[0].driver_nodes)), str(len(matchings[1].driver_nodes)), 
                        str(pre_intersection_size), str(intersection_size), str(pre_union_size), str(union_size)
                    ]) + "\n")

In [ ]:
def diff_k(
        k_range={"start": 1, "end": 8, "step": 0.1}, 
        result_columns=["type", "n", "k1", "k2", "MDS1", "MDS2", "pre_intersection", "intersection", "pre_union", "union"]):
    output_file_name = create_output_file(result_columns)

    n = 1000
    for type, _ in config.NETWORK_TYPES.items():
        for k1 in np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"]):
            k1 = round(k1, 2)
            dir1 = f"{type}_n={n}_k={k1}"
            for k2 in np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"]):
                k2 = round(k2, 2)
                dir2 = f"{type}_n={n}_k={k2}"

                matchings = []
                graph1 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, dir1, "1.txt"), n)
                matching1 = Matching(graph1)
                matching1.HK_algorithm()
                matching1.find_all_alternating_reachable_set()
                matchings.append(matching1)
                graph2 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, dir2, "2.txt"), n)
                matching2 = Matching(graph2)
                matching2.HK_algorithm()
                matching2.find_all_alternating_reachable_set()
                matchings.append(matching2)

                multi_matching = MultiMatching(matchings)
                pre_intersection_size, intersection_size, pre_union_size, union_size = multi_matching.MOUI()
                with open(output_file_name, "a", encoding="utf-8") as output_file:
                    output_file.write(",".join([
                        f"{type}+{type}", str(n), str(k1), str(k2), 
                        str(len(matching1.driver_nodes)), str(len(matching2.driver_nodes)),
                        str(pre_intersection_size), str(intersection_size), str(pre_union_size), str(union_size)
                    ]) + "\n")
    
    for k1 in np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"]):
        k1 = round(k1, 2)
        dir1 = f"ER_n={n}_k={k1}"
        for k2 in np.arange(k_range['start'], k_range['end'] + k_range["step"], k_range["step"]):
            k2 = round(k2, 2)
            dir2 = f"BA_n={n}_k={k2}"

            matchings = []
            graph1 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, dir1, "1.txt"), n)
            matching1 = Matching(graph1)
            matching1.HK_algorithm()
            matching1.find_all_alternating_reachable_set()
            matchings.append(matching1)
            graph2 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, dir2, "1.txt"), n)
            matching2 = Matching(graph2)
            matching2.HK_algorithm()
            matching2.find_all_alternating_reachable_set()
            matchings.append(matching2)

            multi_matching = MultiMatching(matchings)
            pre_intersection_size, intersection_size, pre_union_size, union_size = multi_matching.MOUI()
            with open(output_file_name, "a", encoding="utf-8") as output_file:
                output_file.write(",".join([
                    f"ER+BA", str(n), str(k1), str(k2), 
                    str(len(matching1.driver_nodes)), str(len(matching2.driver_nodes)),
                    str(pre_intersection_size), str(intersection_size), str(pre_union_size), str(union_size)
                ]) + "\n")

In [ ]:
def diff_gamma(
        gamma_range={"start": 2, "end": 8, "step": 0.1}, 
        result_columns=["type", "n", "k", "gamma1", "gamma2", "MDS1", "MDS2", "pre_intersection", "intersection", "pre_union", "union"]):
    output_file_name = create_output_file(result_columns)

    type = "BA"
    n = 1000
    k = round(3.0, 2)
    for gamma1 in np.arange(gamma_range['start'], gamma_range['end'] + gamma_range["step"], gamma_range["step"]):
        gamma1 = round(gamma1, 2)
        dir1 = f"{type}_n={n}_k={k}_gamma={gamma1}"
        for gamma2 in np.arange(gamma_range['start'], gamma_range['end'] + gamma_range["step"], gamma_range["step"]):
            gamma2 = round(gamma2, 2)
            dir2 = f"{type}_n={n}_k={k}_gamma={gamma2}"

            matchings = []
            graph1 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, dir1, "1.txt"), n)
            matching1 = Matching(graph1)
            matching1.HK_algorithm()
            matching1.find_all_alternating_reachable_set()
            matchings.append(matching1)
            graph2 = read_network(os.path.join(config.SYNTHETIC_NET_PATH, dir2, "2.txt"), n)
            matching2 = Matching(graph2)
            matching2.HK_algorithm()
            matching2.find_all_alternating_reachable_set()
            matchings.append(matching2)

            multi_matching = MultiMatching(matchings)
            pre_intersection_size, intersection_size, pre_union_size, union_size = multi_matching.MOUI()
            with open(output_file_name, "a", encoding="utf-8") as output_file:
                output_file.write(",".join([
                    f"{type}+{type}", str(n), str(k), str(gamma1), str(gamma2), 
                    str(len(matching1.driver_nodes)), str(len(matching2.driver_nodes)),
                    str(pre_intersection_size), str(intersection_size), str(pre_union_size), str(union_size)
                ]) + "\n")

In [ ]:
def real_network(
        result_columns=["Dataset", "N1", "N2", "E1", "E2", "MDS1", "MDS2", "t_HK1", "t_HK2", "t_ARS1", "t_ARS2", "t_MSS", "pre_intersection", "intersection", "pre_union", "union"]):
    output_file_name = create_output_file(result_columns)
    base_path = 'assets/real_net2'
    dataset_folders = [f.path for f in os.scandir(base_path) if f.is_dir()]
    for folder in dataset_folders:
        print(folder)

        graph1 = nx.DiGraph()
        graph2 = nx.DiGraph()
        graphs = [graph1, graph2]

        layers_files = glob.glob(os.path.join(folder, 'Dataset', '*_layers.txt'))
        edges_files = glob.glob(os.path.join(folder, 'Dataset', '*_multiplex.edges'))
        nodes_files = glob.glob(os.path.join(folder, 'Dataset', '*_nodes.txt'))

        if not layers_files or not edges_files or not nodes_files:
            print(f"Missing files in dataset {folder}. Skipping...")
            continue

        # 选择第一个文件，如果有多个匹配的话
        layers_file = layers_files[0]
        edges_file = edges_files[0]
        nodes_file = nodes_files[0]

        layers = pd.read_csv(layers_file, sep=' ')

        nodes = pd.read_csv(nodes_file, sep=' ', on_bad_lines='skip')

        edges = pd.read_csv(edges_file, sep=' ', header=None)
        edges.columns = ['layerID', 'node1', 'node2', 'weight']

        # print(f"layers: {layers}, {layers.columns}, {layers.iloc[0, 0]}, {layers.iloc[1, 0]}")
        # print(f"nodes: {nodes}, {nodes.columns}")
        # print(f"edges: {edges}")
        # break

        for graph in graphs:
            graph.add_nodes_from(nodes['nodeID'].tolist())

        edges_layer1 = edges[edges['layerID'] == layers.iloc[0, 0]]
        edges_layer2 = edges[edges['layerID'] == layers.iloc[1, 0]]

        graph1.add_edges_from(edges_layer1[['node1', 'node2']].values.tolist())
        graph2.add_edges_from(edges_layer2[['node1', 'node2']].values.tolist())
        
        matchings = []
        matching1 = Matching(graph1)
        _, hk_time1 = matching1.HK_algorithm()
        _, ARS_time1 = matching1.find_all_alternating_reachable_set()
        matchings.append(matching1)
        matching2 = Matching(graph2)
        _, hk_time2 = matching2.HK_algorithm()
        _, ARS_time2 = matching2.find_all_alternating_reachable_set()
        matchings.append(matching2)

        multi_matching = MultiMatching(matchings)
        [pre_intersection_size, intersection_size, pre_union_size, union_size], MSS_time = multi_matching.MOUI()
        with open(output_file_name, "a", encoding="utf-8") as output_file:
            output_file.write(",".join([
                folder.split("\\")[-1],
                str(graph1.number_of_nodes()), str(graph2.number_of_nodes()), 
                str(graph1.number_of_edges()), str(graph2.number_of_edges()), 
                str(len(matching1.driver_nodes)), str(len(matching2.driver_nodes)),
                str(hk_time1), str(hk_time2), str(ARS_time1), str(ARS_time2), str(MSS_time),
                str(pre_intersection_size), str(intersection_size), str(pre_union_size), str(union_size)
            ]) + "\n")